# Calculate source Luminosity

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import uncertainties as uncer
import uncertainties.umath as umath
from astropy import units as u
from scipy.optimize import curve_fit
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)
%matplotlib widget


In [ ]:
### frequency regime definition
nu_interval = [0, 3.0E11, 7.5E14, 3.0E16, 3.0E19, 3.0E30]

### spectral index functinon
def alpha(x, a, c):
    return x+c*a

### luminosity function
def Luminosity(flux, dL, z, alpha):
    L = flux*1.0E4*4*np.pi*dL**2/(1+z)**(1-alpha)
    return L

###
def ClearData():
    data = {
    'freq': [],
    'L': [],
    'L_err': []
    }
    return data

### read in source list
table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/data/BZCAT_Xmatch_ACC_list.csv')

table_name = table.name.values
table_class = table.Class.values
table_z = table.z.values

### read in all source flux file
path = "/Users/87steven/Documents/ASIAA/Blazar SED code and data/source individual flux 2023_2_15"
dir_list = os.listdir(path) 

dir_list = set(dir_list)
#dir_list.remove('.DS_Store')
dir_list = sorted(dir_list)

for i in range(0, len(dir_list)):   # len(dir_list), 
    df = ClearData()
    
    ### test source 
    #source_name = 'J0304-7007'
    
    ### loop  source
    source_name = dir_list[i][0:10]
    print('i = ', i, ', source name = ', source_name, '(', i ,'/', len(dir_list),')' )
    
    ### constrain in redshift
    name_index  = np.where(source_name == table_name)[0]
    z = table_z[name_index][0]
    
    if z == 0:
        z = 0.01
    
    if z > 0 | ~np.isnan(z):
        ### luminosity distance calculation
        dL_Mpc = cosmo.luminosity_distance(z).value  ### [Mpc]
        dL_m = dL_Mpc*1.0E6*3.08E16 ### [m]

        ### read in flux 
        table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/source individual flux 2023_2_15/' + dir_list[i]) 

        freq = table['freq'].values
        flux = table['flux'].values
        flux_err = table['flux_err'].values   # rekative error

        flux_err = flux*flux_err/0.434   # true error

        ### find index of flux == nan, flux <= 0, flux_err == nan, and flux_err == 0
        index = np.where((~np.isnan(flux)) & (flux > 0) & (~np.isnan(flux_err)) & (flux_err > 0))[0]
        ### exclude points with flux and flux_err == 0 or == nan
        freq = freq[index]
        flux = flux[index]
        flux_err = flux_err[index]

        freq_array = []
        luminosity_array = []
        luminosity_err_array = []
        for k in range(0, len(nu_interval)-1):
            #print('k = ', k, 'to', k+1)

            index = np.where( (nu_interval[k] < freq) & (freq < nu_interval[k+1]) )
            freq_PL_fit = np.log10(freq[index])
            flux_PL_fit = np.log10(flux[index])

            if len(freq_PL_fit) > 2:
                popt, pcov = curve_fit(alpha, freq_PL_fit, flux_PL_fit)

                #print('alpha = ', popt[0])
                freq_array.append(10**freq_PL_fit)
                luminosity_array.append( Luminosity(flux[index], dL_m, z, popt[0]) )
                luminosity_err_array.append(Luminosity(flux_err[index], dL_m, z, popt[0]))

        freq_flat = [item for sublist in freq_array for item in sublist]
        L_flat = [item for sublist in luminosity_array for item in sublist]
        L_err_flat = [item for sublist in luminosity_err_array for item in sublist]

        for m in range(0, len(freq_flat)):
            df['freq'].append(freq_flat[m])
            df['L'].append(L_flat[m])
            df['L_err'].append(L_err_flat[m])

        CSVfile = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/source individual luminosity/' + source_name + '.csv'   
        dff = pd.DataFrame(df)      
        dff.to_csv(CSVfile, index = False)

        print('======================= File saved successfully =======================')
    else:
        print('======================= NO redshift information =======================')
